<a href="https://colab.research.google.com/github/c-susan/Group-Project/blob/main/Group_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Loading Packages**

In [4]:
# Commands to install some of the libraries in-case if they are not installed
# Any other library that needs to be installed just use: !pip install <library name>
# !pip install seaborn
# !pip install missingno
# !pip install xgboost
#!pip install catboost
# !pip install regex
# !pip install sklearn
# !pip install pandas
# !pip install numpy
# !pip install imblearn
# !pip install lightgbm

In [5]:
import pandas as pd
import numpy as np   # linear algebra
import matplotlib.pyplot as plt  #graphs and plots
import seaborn as sns   #data visualizations
import csv # Some extra functionalities for csv  files - reading it as a dictionary
from lightgbm import LGBMClassifier #sklearn is for machine learning and statistical modeling including classification, regression, clustering and dimensionality reduction

from sklearn.model_selection import train_test_split, cross_validate   #break up dataset into train and test sets

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler


# importing python library for working with missing data
import missingno as msno
# To install missingno use: !pip install missingno
import re    # This library is used to perform regex pattern matching

# import various functions from sklearn
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier
from catboost import CatBoostClassifier
import xgboost as xgb
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, classification_report, make_scorer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split

In [6]:
from sklearn.model_selection import KFold,cross_val_score, RepeatedStratifiedKFold,StratifiedKFold
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import OneHotEncoder,StandardScaler,PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer,SimpleImputer
from sklearn.compose import make_column_transformer
from imblearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.impute import SimpleImputer
from sklearn.dummy import DummyClassifier
from imblearn.over_sampling import SMOTE
from sklearn.neighbors import KNeighborsClassifier
#from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score,\
                            #precision_score, recall_score, roc_auc_score,\
                            #plot_confusion_matrix, classification_report, plot_roc_curve, f1_score

import plotly
import plotly.express as px
import plotly.graph_objs as go
import plotly.offline as py
from plotly.offline import iplot
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

import warnings
warnings.filterwarnings("ignore")

# **Data Cleaning**

In [7]:
diabetes = pd.read_csv('diabetic_data.csv')
diabetes.head()

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


## **1. Dropping Unnecessary Columns**

In [8]:
# Dropping the 'encounter_id', 'patient_nbr', 'admission_type_id', 'discharge_disposition_id',
## and 'admission_source_id' columns
diabetes.drop(['encounter_id',
               'patient_nbr',
               'admission_type_id',
               'discharge_disposition_id',
               'admission_source_id'], axis = 1, inplace = True)

## **2. Checking for Duplicates in Data**

In [9]:
diabetes.duplicated().sum()

0

## **3. Checking for Missing Values**

In [10]:
# Some of the missing values are marked with '?'.
## The script below changes the '?' values to NAN values to explore the amount of missing data in each column.
diabetes = diabetes.replace('?', np.nan)

In [11]:
# Checking for amount of null/missing values.
def missing (diabetes):
    missing_number = diabetes.isnull().sum().sort_values(ascending=False)
    missing_percent = ((diabetes.isnull().sum()/diabetes.isnull().count())*100).sort_values(ascending=False)
    missing_values = pd.concat([missing_number, missing_percent], axis=1, keys=['Missing_Number', 'Missing_Percent %'])
    return missing_values

missing(diabetes)

,Missing_Number,Missing_Percent %
weight,98569,96.858479
max_glu_serum,96420,94.746772
A1Cresult,84748,83.277322
medical_specialty,49949,49.082208
payer_code,40256,39.557416
race,2273,2.233555
diag_3,1423,1.398306
diag_2,358,0.351787
diag_1,21,0.020636
glimepiride-pioglitazone,0,0.000000


In [12]:
# According to the variable table/data dictionary, the 'A1Cresult' and 'max_glu_serum' do not have missing values.
# The 'nan' values in two columns indicate that no measurements were taken.
# The script below replaces the 'nan' values of the two columns with 'no measured'.
diabetes[['A1Cresult', 'max_glu_serum']] = diabetes[['A1Cresult', 'max_glu_serum']].fillna('not measured')

In [ ]:
# Visualizing the missing data
msno.bar(diabetes)

<Axes: >

In [ ]:
# Dropping the 'weight' column as it has large percentage (%97) of missing values
diabetes.drop(['weight','diag_1','diag_2','diag_3'], axis = 1, inplace = True)

In [ ]:
# Because the columns with missing values are all categorical values with object datatypes,
# I decided to replace the null values with 'N/A'.
diabetes = diabetes.fillna('N/A')

In [ ]:
msno.bar(diabetes)

# **Metrics of Importance**

In [ ]:
numerical= ['time_in_hospital','num_lab_procedures','num_procedures','num_medications','number_outpatient',
           'number_emergency','number_inpatient','number_diagnoses']

categorical = ['race','gender','age','time_in_hospital','insulin','change','diabetesMed']

In [ ]:
## 0 = not readmitted and readmitted after 30 days
## 1 = readmitted within 30 days
readmitted = {'NO':0, '>30':0, '<30':1}
diabetes['readmitted'] = diabetes['readmitted'].map(readmitted)

In [ ]:
from sklearn.metrics import mutual_info_score
def cat_mut_inf(series):
    return mutual_info_score(series, diabetes['readmitted'])

diabetes_cat = diabetes[categorical].apply(cat_mut_inf)
diabetes_cat = diabetes_cat.sort_values(ascending=False).to_frame(name='mutual_info_score')
diabetes_cat

### **Correlation Matrix**

In [ ]:
diabetes[numerical].corr()

In [ ]:
diabetes.groupby('readmitted')[numerical].mean()

In [ ]:
diabetes[['time_in_hospital','num_medications','number_diagnoses', 'readmitted']].corr()

### **Scatterplots**

**Time in Hospital & Readmitted**

In [ ]:
fig = px.scatter(diabetes, x='time_in_hospital', y='readmitted', title='Time in Hospital & Readmitted',color='readmitted', hover_data = diabetes[['readmitted']])
fig.show()

**Num Medications & Readmitted**

In [ ]:
fig = px.scatter(diabetes, x='num_medications', y='readmitted', title='Number of Medications & Readmitted',color='readmitted', hover_data = diabetes[['readmitted']])
fig.show()

**Num Diagnoses & Readmitted**

In [ ]:
fig = px.scatter(diabetes, x='number_diagnoses', y='readmitted', title='Number of Diagnoses & Readmitted',color='readmitted', hover_data = diabetes[['readmitted']])
fig.show()

# **Converting Categorical Features into Numeric Features**

In [ ]:
# Using get_dummies from the pandas library to convert categorical feature to numeric
diabetes = pd.get_dummies(diabetes, drop_first = False)

# **Breaking the data up into Train & Test**

In [ ]:
train_df, valid_df, test_df = np.split(diabetes.sample(frac=1, random_state=42),
                                       [int(.7*len(diabetes)), int(0.85*len(diabetes))])
train_df = train_df.reset_index(drop = True)
valid_df = valid_df.reset_index(drop = True)
test_df = test_df.reset_index(drop = True)

In [ ]:
diabetes.readmitted.value_counts()

In [ ]:
train_df.readmitted.value_counts()

In [ ]:
valid_df.readmitted.value_counts()

In [ ]:
test_df.readmitted.value_counts()

# Treating the Imbalance in the Data